# 1 Modelo - Ana

#

In [4]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix
from xgboost import XGBClassifier

In [5]:
df_tratado = pd.read_csv("ocorrencias_aereas_tratadas.csv", encoding='latin1', sep=',')
display(df_tratado)

,codigo_ocorrencia,ocorrencia_classificacao,ocorrencia_latitude,ocorrencia_longitude,ocorrencia_cidade,ocorrencia_uf,ocorrencia_pais,ocorrencia_aerodromo,ocorrencia_dia,ocorrencia_hora,...,aeronave_voo_destino,aeronave_tipo_operacao,ocorrencia_ano,idade,outlier_ocorrencia_latitude,outlier_ocorrencia_longitude,outlier_aeronave_pmd,outlier_aeronave_assentos,outlier_aeronave_ano_fabricacao,cluster
0,86954,ACIDENTE,-29.085556,-29.085556,TUPANCIRETÃÂ,RS,BRASIL,SIPJ,04/04/2025,17:20:00,...,SANTA MARIA,PRIVADA,2025,35.0,normal,normal,normal,normal,normal,0
1,86953,ACIDENTE,-9.596944,-9.596944,PILAR,AL,BRASIL,FAER,04/04/2025,09:05:00,...,FORA DE AERODROMO,AGRÃÂCOLA,2025,15.0,normal,normal,normal,normal,normal,0
2,86952,INCIDENTE,-2.586944,-2.586944,SÃÂO LUÃÂS,MA,BRASIL,SBSL,03/04/2025,20:45:00,...,MARECHAL CUNHA MACHADO,EXPERIMENTAL,2025,11.0,normal,normal,normal,normal,normal,0
3,86951,INCIDENTE GRAVE,-27.133889,-27.133889,CHAPECÃÂ,SC,BRASIL,SBCH,31/03/2025,22:30:00,...,SERAFIN ENOSS BERTASO,REGULAR,2025,14.0,normal,normal,normal,normal,normal,1
4,86950,INCIDENTE,-23.435556,-23.435556,GUARULHOS,SP,BRASIL,SBGR,05/04/2025,11:00:00,...,GOVERNADOR ANDRÃÂ FRANCO MONTORO,REGULAR,2025,4.0,normal,normal,normal,normal,normal,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13145,28377,ACIDENTE,-5.460139,-5.460139,IMPERATRIZ,MA,BRASIL,FAER,27/01/2007,11:18:00,...,COMANDANTE CARLOS INÃÂCIO AGNES,PRIVADA,2007,37.0,normal,normal,normal,normal,normal,0
13146,28375,ACIDENTE,-23.612222,-23.612222,APUCARANA,PR,BRASIL,FAER,26/01/2007,18:15:00,...,CONGONHAS,TÃÂXI AÃÂREO,2007,1.0,normal,normal,normal,normal,normal,0
13147,28355,ACIDENTE,-17.254167,-17.254167,BOCAIÃÂVA,MG,BRASIL,FAER,18/01/2007,09:10:00,...,MÃÂRIO RIBEIRO,TÃÂXI AÃÂREO,2007,41.0,normal,normal,normal,normal,normal,0
13148,28335,ACIDENTE,-23.344444,-23.344444,LONDRINA,PR,BRASIL,FAER,19/01/2007,12:30:00,...,GOVERNADOR JOSÃÂ RICHA,PRIVADA,2007,21.0,normal,normal,normal,normal,normal,0


In [7]:
# Defina a coluna alvo (substitua 'target' pelo nome correto)
target_col = 'ocorrencia_classificacao' 

# Separe features e alvo
X = df_tratado.drop(columns=[target_col])
y = df_tratado[target_col]

# Se necessário, transforme variáveis categóricas em dummies
X = pd.get_dummies(X)

# Divida em treino e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=0.2, random_state=42)

# 1. Random Forest
rf = RandomForestClassifier(class_weight='balanced', random_state=42)
rf.fit(X_train, y_train)
y_pred_rf = rf.predict(X_test)
print("Random Forest:")
print(classification_report(y_test, y_pred_rf))
print(confusion_matrix(y_test, y_pred_rf))

Random Forest:
                 precision    recall  f1-score   support

       ACIDENTE       0.71      0.80      0.75       568
      INCIDENTE       0.89      0.96      0.92      1826
INCIDENTE GRAVE       0.28      0.03      0.05       236

       accuracy                           0.84      2630
      macro avg       0.63      0.60      0.58      2630
   weighted avg       0.80      0.84      0.81      2630

[[ 457  104    7]
 [  71 1744   11]
 [ 115  114    7]]


In [ ]:
# 2. Regressão Logística
lr = LogisticRegression(class_weight='balanced', max_iter=1000, random_state=42)
lr.fit(X_train, y_train)
y_pred_lr = lr.predict(X_test)
print("Logistic Regression:")
print(classification_report(y_test, y_pred_lr))
print(confusion_matrix(y_test, y_pred_lr))

KeyboardInterrupt: 

In [10]:
# Calcula o peso da classe negativa em relação à positiva (ou vice-versa)
from collections import Counter

counts = Counter(y_train)
scale = counts[0] / counts[1] if counts[0] > counts[1] else counts[1] / counts[0]

model_xgb = XGBClassifier(scale_pos_weight=scale, use_label_encoder=False, eval_metric='logloss', random_state=42)
model_xgb.fit(X_train, y_train)
y_pred_xgb = model_xgb.predict(X_test)

print("XGBoost:\n", classification_report(y_test, y_pred_xgb))

ZeroDivisionError: division by zero

In [12]:
from lightgbm import LGBMClassifier

model_lgb = LGBMClassifier(class_weight='balanced', random_state=42)
model_lgb.fit(X_train, y_train)
y_pred_lgb = model_lgb.predict(X_test)

print("LightGBM:\n", classification_report(y_test, y_pred_lgb))

LightGBMError: Do not support special JSON characters in feature name.